Treat image ROI takes electron diffraction patterns and calibrate the image, rotate it, draw the BZ around the Bragg peak and average equivalent BZ together

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import matplotlib.ticker as ticker
import cv2  # importing cv
from skimage import img_as_float
from skimage import filters
import functions_analysis as fa
import find_peak

### All folder should countain a folder named 'data' containing pickle files ###

scan_number = '800nm'
path =r'C:\Users\rclaude\Documents\projet\graphite\data_UED'

t0_num = 8 #start from one

Zorder_in = np.array([242,247]) # guess position 0th order
BP1 = np.array([147,234]) # guess position 1st order Bragg peak
BP2 = np.array([53,222]) # guess position 2nd order Bragg peak
method = '2D_voigt' ## method for fit

df = 0
file_list = (glob.glob(path + '/RAW_sorted/RAW_' + scan_number + '*.pickle'))
print('list of detected files : ', file_list)
imgON = []
imgOFF = []
for file in file_list: 
    input_df = pd.read_pickle(file, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
    imgON.append(np.stack(input_df['imagesON']))
    imgOFF.append(np.stack(input_df['imagesOFF']))

imgON_raw = np.sum(np.array(imgON), axis=0)
imgOFF_raw = np.sum(np.array(imgOFF), axis=0)
imgON = np.copy(imgON_raw)
imgOFF = np.copy(imgOFF_raw)
print('shape of image : ', imgON.shape)

df1 = input_df.drop('imagesON', axis=1)
df1 = input_df.drop('imagesOFF', axis=1)
delay= (df1['LTS_position'].to_numpy())
delay = (delay-delay[t0_num-1])*6.666
n_delay = len(delay)
print('delay array : ', delay)


Find threshold count

In [ ]:
s_roi=15
ROI = [Zorder_in[1]-s_roi, Zorder_in[1]+s_roi, Zorder_in[0]-s_roi, Zorder_in[0]+s_roi]
k = find_peak.make_ROI(imgOFF[0], *ROI)
thresh = (np.max(k))*1.01
print('Hot pixel threshold : ', thresh)


Get rid of the hot pixel : set them with OFF pixels value

In [ ]:
fig, axiss = plt.subplots(1,2, figsize=(10, 5), layout='tight')
axis = axiss[0]
ax =axiss[1]
axis.hist(((imgON/imgOFF)[0]).flatten(), bins=100, label=r'original pixel distribution')
axis.set_yscale('log')

ax.hist(imgON[0].flatten(), bins=100, label = 'ON before')
ax.hist(imgOFF[0].flatten(), bins=100, label = 'OFF before')

ax.set_yscale('log')

r_1 = 0.8
r_2 = 1.15

## remove hot pixel
imgON, imgOFF, rh_abs = fa.remove_hot_abs(imgON, imgOFF, thresh, True)
imgON, imgOFF, rh = fa.remove_hot_uneven(imgON, imgOFF,r_1, r_2, True)



print('Pixel ratio above threshold: ', rh_abs)
print('Pixel ratio defect: ', rh_abs)

axis.hist(((imgON/imgOFF)[0]).flatten(), bins=100, label=r'pixel disitribution after removing %.3f percent'%rh)
ax.hist(imgON[0].flatten(), bins=100, label = 'ON after')
ax.hist(imgOFF[0].flatten(), bins=100, label = 'OFF after')
ax.legend()
ax.set_yscale('log')
axis.set_yscale('log')
axis.set_xlim(0,2)
axis.legend()
axis.set_title(r'Removing pixel between %.2f and %.2f'%(r_1, r_2))
ax.set_title(r'Removing pixel above %.3e'%thresh)



Look at the position of the unscaterred beam with different fitting method 

In [ ]:
methods = [ '2D_gaussian', '2D_lorentzian',  'pseudo-voigt','2D_voigt','2D_voigt_rotated']
s_roi=15

ROI = [Zorder_in[1]-s_roi, Zorder_in[1]+s_roi, Zorder_in[0]-s_roi, Zorder_in[0]+s_roi]
o=0
print('with ROI %i x %i'%(s_roi, s_roi))
for met in methods: 
    pos_on = find_peak.get_pos(imgON[o], ROI, method=met, show_plot=False)
    pos_off = find_peak.get_pos(imgOFF[o], ROI, method=met)
    print('from '+met+' pos on = [%.3f, %.3f]'%(pos_on[0], pos_on[1]), ' and pos off = [%.3f, %.3f]'%(pos_off[0], pos_off[1]))
    print('difference = [%.2e, %.2e]'%(pos_on[0]-pos_off[0], pos_on[1]-pos_off[1]))


Get the right position of the 0th order and the 1st order Bragg peak 

In [ ]:
s_roi1 = 15
s_roi2 = 15
ROI = [Zorder_in[1]-s_roi1, Zorder_in[1]+s_roi1, Zorder_in[0]-s_roi2, Zorder_in[0]+s_roi2]
## Zorder is the the position of the unscattered beam for delay = 0 
Zorder = (find_peak.get_pos(imgON[-1], ROI, method, True ))


Zorder_in = np.round(Zorder).astype(np.int16)

BP_init = (find_peak.get_pos_around(imgON[0], BP1, 10, method, True ))
BP_in = np.round(BP_init).astype(np.int16)

BP_init_2 = (find_peak.get_pos_around(imgON[0], BP2, 10, method, True ))
BP_in_2 = np.round(BP_init_2).astype(np.int16)

print('position of the 0th order : ', Zorder, ' and as integer : ',  Zorder_in, ' fitting with : ', method)


print('position of the initial 1st order Bragg peak : ', BP_init, ' and as integer : ',  BP_in, ' fitting with : ', method)
print('position of the initial 2nd order Bragg peak : ', BP_init_2, ' and as integer : ',  BP_in_2, ' fitting with : ', method)


vec_i = Zorder - BP_init 
vec_i_2 = Zorder - BP_init_2
angle_sym = np.pi/2

Get the position of all the 1st order Bragg peak with the symmetry 

In [6]:
s_roi=5
angle_sym=np.pi/3
sym_ax=6
BP_a = []
BP_b = [] 
for i in range(sym_ax):
    BP_a.append(Zorder + fa.rotate_vector(vec_i, i*angle_sym))
    BP_b.append(Zorder + fa.rotate_vector(vec_i_2, i*angle_sym))


BP_a = np.array(BP_a).astype(np.int32) 
BP_b = np.array(BP_b).astype(np.int32) 

Shift the 0th order 

In [ ]:
center0_on = []
center0_off = []
# method='pseudo-voigt'
s_roi=15
ROI = [Zorder_in[1]-s_roi, Zorder_in[1]+s_roi, Zorder_in[0]-s_roi, Zorder_in[0]+s_roi]
for i in range(len(delay)):
    center0_on.append(find_peak.get_pos(imgON[i], ROI, method))
    center0_off.append(find_peak.get_pos(imgOFF[i], ROI, method))
s=0.1
center0_on = np.array(center0_on)
center0_off = np.array(center0_off)

### use function shift_0order in functions_analysis 
### We first shift image ON 0th order beam position to the position of imageOFF
### Then we shift both imgON and imgOFF to a reference point 
imgON_shifted = fa.shift_0order_fit(imgON, imgOFF, Zorder_in, s_roi, method)

ref = find_peak.get_pos(np.sum(imgOFF, axis=0), ROI, method)
print('The reference point is :', ref)

imgON_shifted = fa.shift_to_ref(imgON_shifted, ref, Zorder_in, s_roi, method)
imgOFF_shifted = fa.shift_to_ref(imgOFF, ref, Zorder_in, s_roi, method)
print(np.shape(imgON_shifted))

center0_on_shifted = []
center0_off_shifted = []

for i in range(len(delay)):
    center0_on_shifted.append(find_peak.get_pos(imgON_shifted[i], ROI, method))
    center0_off_shifted.append(find_peak.get_pos(imgOFF_shifted[i], ROI, method))

center0_on_shifted = np.array(center0_on_shifted)
center0_off_shifted = np.array(center0_off_shifted)


print('for the fist delay, discrepancy between on and off is : ', center0_on[0,0]-center0_off[0,0])
print('Zero order before shifting : ', center0_on[0])
print('Zero order after shifting : ', center0_on_shifted[0])
ind = -1
fig, ax = plt.subplots(2,2, figsize=(10,10))
fig.suptitle('fit with ' + method)
ax[0][0].set_title('deviation ON-OFF along axis')
ax[0][0].plot(delay, center0_on[:,0]-center0_off[:,0], label='along x')
ax[0][0].plot(delay, center0_on[:,1]-center0_off[:,1], label='along y')
# ax[0][0].plot(delay, center0_on_shifted[:,0]-center0_off[:,0], label='along x shifted')
# ax[0][0].plot(delay, center0_on_shifted[:,1]-center0_off[:,1], label='along y shifted')
ax[0][0].legend()
ax[0][0].set_xlabel('time (ps)')
ax[0][0].set_xlabel('deviation (px)')
img = imgON[ind, Zorder_in[0]-s_roi:Zorder_in[0]+s_roi, Zorder_in[1]-s_roi:Zorder_in[1]+s_roi] / imgOFF[ind, Zorder_in[0]-s_roi:Zorder_in[0]+s_roi,Zorder_in[1]-s_roi: Zorder_in[1]+s_roi] 
ax[0][1].set_title(r'ON/OFF($t=%.1f$) ps without shifting correction'%delay[ind])
ax[0][1].imshow(img-1, cmap='bwr', vmin=-s, vmax=s)

ax[1][0].set_title(r'norm of the deviation')
ax[1][0].plot(delay, np.linalg.norm(center0_on, axis=1), '+-', label='ON')
ax[1][0].plot(delay, np.linalg.norm(center0_off, axis=1), '+-', label='OFF')

ax[1][0].plot(delay, np.linalg.norm(center0_on_shifted, axis=1), label='ON shifted')
ax[1][0].plot(delay, np.linalg.norm(center0_off_shifted, axis=1), label='OFF shifted')
ax[1][0].set_xlabel('time (ps)')
ax[1][0].set_xlabel('deviation (px)')
ax[1][0].legend()
img_shift = imgON_shifted[ind, Zorder_in[0]-s_roi:Zorder_in[0]+s_roi, Zorder_in[1]-s_roi:Zorder_in[1]+s_roi] / imgOFF_shifted[ind, Zorder_in[0]-s_roi:Zorder_in[0]+s_roi,Zorder_in[1]-s_roi: Zorder_in[1]+s_roi]
ax[1][1].set_title(r'ON/OFF($t=%.1f$ ps) with shifting correction'%(delay[ind]))
ax[1][1].imshow(img_shift-1, cmap='bwr', vmin=-s, vmax=s)


Get the distance between 1st order Bragg peak and the angle between the vectors 

In [ ]:
### suming over all delay 
s_roi=10
vecON_a, vecOFF_a = fa.get_distance_on_off(np.sum(imgON_shifted, axis=0), np.sum(imgOFF_shifted, axis=0), [BP_a[0], BP_a[3]], s_roi, '2D_voigt', False)
s_roi=20
vecON_b, vecOFF_b = fa.get_distance_on_off(np.sum(imgON_shifted, axis=0), np.sum(imgOFF_shifted, axis=0), [BP_b[0], BP_b[3]], s_roi, '2D_voigt', False)

print('along a*, the distance for pump on = %.3f px and pump off = %.3f px'%(np.linalg.norm(vecON_a), np.linalg.norm(vecOFF_a)) )
print('along b*, the distance for pump on = %.3f px and pump off = %.3f px'%(np.linalg.norm(vecON_b), np.linalg.norm(vecOFF_b)) )
print('angle between 1st and 2nd order BP when on = %.4f and when off = %.4f'%(fa.angle_between(vecON_a,vecON_b)*180/np.pi, fa.angle_between(vecOFF_a,vecOFF_b)*180/np.pi))



Average between output from the two above cells. compute the calibration constant based on L and theoretical value. 
OUTPUT : alpha, L, cal, middle

In [ ]:
vert = np.array([0,1])
alpha = fa.angle_between(vecON_a, vert)*180/np.pi-90
La = (np.linalg.norm(vecON_a)+np.linalg.norm(vecOFF_a))/4
Lb=  (np.linalg.norm(vecON_b)+np.linalg.norm(vecOFF_b))/8
L = (La+Lb)/2
print('angle of form vertical: %.3f'%(alpha) )
print('distance between peak along a* = %.3f px'%(La))
print('distance between peak along b* = %.3f px'%(Lb))
print('mean distance = %.3f px'%L)
a = 2.46 
g_px = L
g_A = 4*np.pi/(np.sqrt(3)*a)
cal = g_A/g_px ## A-1/px
# cal = 0.031112607906503217

print('mean distance between the peaks: %.3f px'%L)
print('calibration constant from graphite : %.3e 1/A/px'%cal)
print('reciprocal lattice constant from cal : %.3e 1/A'%(L*cal))
print('lattice constant calibrated a = %.3f px'%(4*np.pi/(La*cal)))
print('lattice constant calibrated b = %.3f px'%(4*np.pi/(Lb*cal)))

middle = np.array([256, 256])

Built the diffraction patterns rotated with respect to the 0th order position 

In [10]:
center = (Zorder[1],Zorder[0])
#rotation matrix for the 6 fold symmetry
rotation_matrix = []    
for i in range(5):
    rotation_matrix.append( cv2.getRotationMatrix2D(center, alpha + i*60, 1.0))


## built array of the rotated diffraction patterns 
img_ON_rotated = [] 
img_OFF_rotated = []
for rot in rotation_matrix:
    img_ON_rotated_one = []
    img_OFF_rotated_one = []

    for j in range(n_delay):
        img_ON_rotated_one.append(cv2.warpAffine(imgON_shifted[j], rot, (imgON_shifted[j].shape[1], imgON_shifted[j].shape[0]), flags=cv2.INTER_LINEAR ))
        img_OFF_rotated_one.append(cv2.warpAffine(imgOFF_shifted[j], rot, (imgOFF_shifted[j].shape[1], imgOFF_shifted[j].shape[0]),flags=cv2.INTER_LINEAR  ))
    
    img_ON_rotated.append(np.array(img_ON_rotated_one))
    img_OFF_rotated.append(np.array(img_OFF_rotated_one))


img_ON_rotated = np.array(img_ON_rotated)
img_OFF_rotated = np.array(img_OFF_rotated)



In [ ]:
### set the misinterpolate pixels to NaN
img_ON_rotated[img_ON_rotated<100] = np.nan
img_OFF_rotated[img_OFF_rotated<100] = np.nan

mask = np.isnan(img_ON_rotated) | np.isnan(img_OFF_rotated)
img_ON_rotated[mask]=np.nan
img_OFF_rotated[mask]=np.nan

## averaged over all symmetry axis
imgON_rot_avg = np.nanmean(img_ON_rotated, axis=0)
imgOFF_rot_avg = np.nanmean(img_OFF_rotated, axis=0)



Look at the position of the 0th order for the different step

In [ ]:
plot = False
print(method)
Zorder_rotated = (find_peak.get_pos(img_ON_rotated[1][0], ROI, method, plot ))
Zorder_rot_averaged = (find_peak.get_pos(imgON_rot_avg[0], ROI, method, plot ))


print('before doing the rotation Zorder is : ', Zorder)
print('Zorder of the rotated image by pi/6 : ', Zorder_rotated)
print('Zorder of averaged along symmetry : ', Zorder_rot_averaged)


Save the data in pickle file 

In [13]:
## filter the image
s=0
imgON_fliped = filters.gaussian(img_as_float(imgON_rot_avg), sigma=s)
imgOFF_fliped = filters.gaussian(img_as_float(imgOFF_rot_avg), sigma=s)


df = pd.DataFrame()
df['imgON'] = list(imgON_fliped)
df['imgOFF'] = list(imgOFF_fliped)
df['delay'] = delay
df.attrs['t0_index'] = t0_num
df.attrs['calibration'] = cal
df.attrs['peak_dist'] = L
df.attrs['scan_number'] = scan_number
df.attrs['Zorder_pos'] = Zorder_rot_averaged


df.to_pickle(path + r'\PROCESSED\PROC_'+scan_number, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

Plot the result 

In [ ]:
at = t0_num
bt = t0_num
%matplotlib inline
print('delay before t0 = ', delay[:bt])
print('delay after t0 = ', delay[at:])

img_diff_raw = (np.mean(imgON[at:], axis=0))/np.mean(imgOFF[at:], axis=0) - (np.mean(imgON[:bt], axis=0))/np.mean(imgOFF[:bt], axis=0)


img_diff_proccessed = (np.mean(imgON_rot_avg[at:], axis=0))/np.mean(imgOFF_rot_avg[at:], axis=0) - (np.mean(imgON_rot_avg[:bt], axis=0))/np.mean(imgOFF_rot_avg[:bt], axis=0)


sens=0.01
sens2 = np.percentile(imgON[0], 99.8)
fig, axis = plt.subplots(1,4, figsize=(20, 6),layout='tight')
axis[0].imshow(np.mean(imgON[at:], axis=0), cmap='bwr',vmin=0, vmax=sens2, aspect=1)
axis[0].set_title('original image')


axis[1].imshow((np.mean(imgON_rot_avg[at:], axis=0)), cmap='bwr',vmin=0, vmax=sens2, aspect=1)
axis[1].set_title('averaged over rotated')


axis[2].imshow(img_diff_raw, cmap='bwr',vmin=-sens, vmax=sens, aspect=1)
axis[2].set_title('ON/OFF original')


axis[3].imshow(img_diff_proccessed, cmap='bwr',vmin=-sens, vmax=sens, aspect=1)
axis[3].set_title('ON/OFF original')


for ax in axis:
    ax.set_xticks([])
    ax.set_yticks([])
